# **Ray: Parallel & Distributed Computing**

In [6]:
%load_ext watermark
%watermark -a 'NavinKumarMNK' -d -t -g -r -u -v -p pytorch_lightning -p torch

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: NavinKumarMNK

Last updated: 2023-02-12 15:31:25

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.8.0

torch: 1.13.1

Git hash: e3b1521eccb3351f0a46358daee37af31b62f403

Git repo: https://github.com/NavinKumarMNK/MegNav.git



## **Ray Lightning**

In [15]:
import pytorch_lightning as pl
from ray_lightning import RayStrategy # DDP

In [16]:
strategy = RayStrategy(num_workers=4, num_cpus_per_worker=1, use_gpu=True)

In [17]:
ptl_model = MNISTClassifier(...)
trainer = pl.Trainer(..., strategy=strategy)
trainer.fit(ptl_model)

NameError: name 'MNISTClassifier' is not defined

In [19]:
import ray.tune as tune
print(tune.choice([16, 32, 64]))

In [20]:
from ray import air, tune
from ray_lightning import RayStrategy
from ray_lightning.tune import TuneReportCallback


In [21]:
def train_mnist(config):
    model = MNISTClassifier(config)
    metrics = {"loss": "ptl/val_loss", "acc": "ptl/val_accuracy"}
    callbacks = [TuneReportCallback(metrics, on="validation_end")]

    trainer = pl.Trainer(
        max_epochs=4,
        callbacks=callbacks,
        plugins=[RayStrategy(num_workers=4, use_gpu=False)])
    trainer.fit(model)

In [22]:
config = {
    "layer_1": tune.choice([32, 64, 128]),
    "layer_2": tune.choice([64, 128, 256]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([32, 64, 128]),
}

In [25]:
tuner = tune.Tuner(
    tune.with_resources(train_mnist, {"cpu": 1, "extra_cpu": 4}),
    tune_config=tune.TuneConfig(
        metric="loss",
        mode="min",
        num_samples=32,
    ),
    param_space=config
)


In [ ]:
results = tuner.fit()
print("Best hyperparameters found were: ", results.get_best_result().config)